In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My Drive/Colab Notebooks/tony_chain

/content/drive/My Drive/Colab Notebooks/tony_chain


In [ ]:
ls

app_by_tony       local_model/      sembot.py          vectorstore_manager.py
chain_logging.py  model_manager.py  sembot_test.ipynb  vector_store_path_jhgan/
config.py         __pycache__/      utils.py


In [ ]:
!pip install transformers langchain huggingface_hub langchain-community langchain-huggingface langchain_openai fastapi uvicorn pypdf accelerate python-dotenv faiss-gpu pdfplumber pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB

In [ ]:
!ngrok authtoken 2oQD6abMRFtGqvuk0x3rONRKN9M_YvTRpFjyQ7NTeeeNJXyB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from sembot import Sembot
from prepare_sembot import SembotDocs

pre_sembot = SembotDocs()
sembot = Sembot()

In [ ]:
model = sembot.get_model()
streamer = sembot.get_streamer()

In [ ]:
# --------------------- FastAPI 설정 부분 --------------------- #

from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import asyncio
from pydantic import BaseModel
from typing import List, Dict, Any
import nest_asyncio
from pyngrok import ngrok
from threading import Thread

app = FastAPI()

# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 보안상 특정 도메인으로 제한할 예정
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["X-Request-ID"],  # 요청 ID를 노출하도록 설정
)

class SearchRequest(BaseModel):
    question: str

class QueryRequest(BaseModel):
    memory: List[Dict[str, Any]]
    question: str

async def async_stream_response(input_data: dict):
    # stream 출력
    # Thread 객체를 사용하기 때문에 동시성 이슈를 신경써야함
    prompt = pre_sembot.make_prompt(input_data)

    generation_kwargs = sembot.inference(prompt)

    thread = Thread(target=model.generate, kwargs=generation_kwargs)

    thread.start()

    for new_text in streamer:
        # print가 아니라 프론트로 전송하는 걸로 바꾸면 될 듯
        yield new_text.replace("\n", "<br>").replace("<|eot_id|>", "")
        await asyncio.sleep(0.01)  # 스트리밍 중 각 청크 사이의 딜레이 추가

    thread.join()


@app.post("/search")
async def search_documents(query: SearchRequest):
    docs = pre_sembot.docs_invoke(query.question)

    return {"docs": docs}

@app.post("/generate")
async def stream_answer(query: QueryRequest):
    input_data = {
        "memory": query.memory,
        "question": query.question
    }

    return StreamingResponse(async_stream_response(input_data), media_type="text/plain")


# 서버 실행
ngrok_tunnel = ngrok.connect(8001)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8001)

INFO:     Started server process [750]
INFO:     Waiting for application startup.


Public URL: https://3646-34-125-160-79.ngrok-free.app
Loading existing Local Model...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     211.192.252.28:0 - "OPTIONS /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "OPTIONS /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "POST /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "POST /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "POST /generate HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
ERROR:__main__:Streaming error: 'TextIteratorStreamer' object has no attribute 'empty'
ERROR:__main__:Streaming error: 'TextIteratorStreamer' object has no attribute 'empty'


INFO:     211.192.252.28:0 - "POST /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "POST /generate HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
ERROR:__main__:Streaming error: 'TextIteratorStreamer' object has no attribute 'empty'
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [750]
